In [40]:
!pip install nltk
!pip install spacy
!python -m spacy download pt_core_news_sm

2023-08-19 17:03:55.670684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 17:03:56.903282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 27.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [41]:
"""
João Assalim
https://www.kaggle.com/datasets/augustop/portuguese-tweets-for-sentiment-analysis?select=NoThemeTweets.csv
"""

'\nJoão Assalim\nhttps://www.kaggle.com/datasets/augustop/portuguese-tweets-for-sentiment-analysis?select=NoThemeTweets.csv\n'

In [42]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score

import string

In [43]:
df = pd.read_csv('./olist.csv')
df = df.drop(["review_text", "review_text_tokenized", "polarity", "kfold_rating", "kfold_polarity", "original_index"], axis=1)
df = df[:40000]
df.head()

,review_text_processed,rating
0,perfeito....chegou antes do prazo.....,5
1,foi uma otima compra! chegou antes mesmo do pr...,5
2,recebi muito rapido e um otimo custo beneficio,5
3,recomendo,5
4,so veio uma capa comprei 3 ai paguei. mais de ...,1


In [44]:
df["rating"] = np.where(df["rating"] < 3, 0, 1)

In [45]:
df.shape

(40000, 2)

In [46]:
df.isnull().sum()

review_text_processed    1
rating                   0
dtype: int64

In [47]:
df = df.dropna()

In [48]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [49]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39999 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   review_text_processed  39999 non-null  object
 1   rating                 39999 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 937.5+ KB


In [51]:
tweets = df["review_text_processed"]
tokenization = [word_tokenize(text.lower()) for text in tweets]

In [52]:
lemmatizer = WordNetLemmatizer()

new_tweets = []

for phrase in tokenization:
  new_phrase = ""
  for token in phrase:
    if not str(token) in stop_words and not token in string.punctuation and "@" not in token and "http" not in token and len(token) > 1 and not token.isdigit():
      new_phrase += lemmatizer.lemmatize(str(token)) + " "
  new_tweets.append(new_phrase[:-1])

In [53]:
df["review_text_processed"] = new_tweets

In [54]:
vect_uni_idf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, norm='l2', stop_words=stop_words)
text_vect_uni_idf = vect_uni_idf.fit_transform(df["review_text_processed"])

X_trainUIDF, X_testUIDF, y_trainUIDF, y_testUIDF = train_test_split(text_vect_uni_idf, df["rating"], test_size=0.2, random_state=42)

In [55]:
rfcidf = RandomForestClassifier()

rfcidf.fit(X_trainUIDF, y_trainUIDF)
y_predUIDF = rfcidf.predict(X_testUIDF)

In [56]:
import joblib
model_filename = 'sentiment_analyser_model.pkl'
joblib.dump(rfcidf, model_filename)


['sentiment_analyser_model.pkl']

In [57]:
model_filename = 'sentiment_analyser_vect.pkl'
joblib.dump(vect_uni_idf, model_filename)

['sentiment_analyser_vect.pkl']